**420-A58-SF - Algorithmes d'apprentissage non supervisé - Hiver 2023 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2023 Mikaël Swawola
<br/>
![Travaux Pratiques - Recherche de documents](static/02-02-A1-banner.png)
<br/>
**Objectif: Lors de l'exploration d'un jeu de données constitué de documents textes - tels que des pages Wikipedia, des articles de presse, StackOverflow, etc., il est courant de chercher à trouver quels sont les documents similaires. L'objectif de cet exercice est de mettre en oeuvre les techniques de recherche adaptées (ici les plus proches voisins) à ce type de données. Les documents utilisés sont les pages Wikipedia de personnalités.**

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

# Le reste des modules sera importé au fur et à mesure des exercices ...

L'archive `people.zip` contient 4 fichiers:

* **people_wiki.csv**: jeu de données consituté des pages Wikipedia de personnalités
* **people_wiki_map_index_to_word.json**: mapping entre les mots et les indices
* **people_wiki_word_count.npz**: vecteurs d'occurence des mots (word count / sacs de mot) pour chaque document
* **people_wiki_tf_idf.npz**: vecteurs TF-IDF pour chaque document

Dans l'énoncé de ce TP, les mots "article" et "document" sont interchangeables.

## 1 - Chargement du jeu de données

**Exercice 1-1 - À l'aide de la librairie Pandas, lire le fichier de données `people/people_wiki.csv`. Afin de permettre les opérations de type `join` effectuées plus loin dans le TP, nommez l'index de la trame de donnée `id`**

In [3]:
# Compléter cette cellule ~ 2 lignes de code

In [4]:
wiki = pd.read_csv('../../data/people/people_wiki.csv')
wiki.index.name = 'id'

**Exercice 1-2 - Afficher les 5 premières lignes de la trame de données. Quelles informations contiennent les colonnes ?**

In [5]:
# Compléter cette cellule ~ 1 ligne de code

In [6]:
wiki.head()

,URI,name,text
id,,,
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


## 2 - Extraction du nombre de mots

Les vecteurs d'occurence des mots (**word count**) du jeu de données ont été préalablement extrait dans le fichier `people/people_wiki_word_count.npz`. Ces vecteurs sont regroupés dans une matrice diluée (sparse), où la i-ème ligne donne le vecteur d'occurence des mots pour le i-ème document. Chaque colonne correspond à un mot unique apparaissant dans le jeu de données. Le mapping entre les mots et les indices est donné dans `people/people_wiki_map_index_to_word.json`

La fonction suivante permet le chargement des vecteurs d'occurence des mots:

In [7]:
from scipy.sparse import csr_matrix

def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

La fonction ci-dessus utilise `csr_matrix` de la bibliothèque SciPy:<br/>
[class scipy.sparse.csr_matrix(arg1, shape=None, dtype=None, copy=False)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)

**Exercice 2-1 - À l'aide de la fonction ci-dessus, charger le ficher contenant les vecteurs d'occurence des mots**

In [8]:
# Compléter cette cellule ~ 1 ligne de code

In [9]:
from scipy.sparse import csr_matrix

word_count = load_sparse_csr('../../data/people/people_wiki_word_count.npz')

**Exercice 2-2 - En vous référant à la documentation de la fonction `csr_matrix`, convertissez la matrice `word_count` en tableau NumPy. Que constatez-vous ?**

In [10]:
# Compléter cette cellule ~ 1 ligne de code
word_count
59071*547979*8

258957340072

In [11]:
word_count.toarray()

MemoryError: Unable to allocate 241. GiB for an array with shape (59071, 547979) and data type int64

**Exercice 2-3 - À l'aide du module json ou de la librairie Pandas, charger le ficher contenant le mapping entre les mots et les indices. Combien y a-t-il de mots dans le dictionnaire ?**

In [12]:
# Compléter cette cellule ~ 2-3 lignes de code

In [13]:
import json

with open('../../data/people/people_wiki_map_index_to_word.json') as f:
     map_index_to_word = json.load(f)
len(map_index_to_word)

547979

In [14]:
map_index_to_word

{'biennials': 522004,
 'lb915': 116290,
 'shatzky': 127647,
 'woode': 174106,
 'damfunk': 133206,
 'nualart': 153444,
 'hatefillot': 164111,
 'missionborn': 261765,
 'yeardescribed': 161075,
 'theoryhe': 521685,
 'vinalop': 222759,
 'soestdijk': 166345,
 'boncea': 150371,
 'spiders': 519990,
 'bienniale': 429277,
 'woody': 541515,
 'trawling': 189895,
 'pampoulovawagner': 201040,
 'bentara': 202586,
 'laserbased': 25758,
 'caner': 346073,
 'canes': 478262,
 'canet': 436468,
 'iaspark': 395341,
 'categoriesborn': 12586,
 '5982': 277649,
 'caney': 459867,
 'phosphorushe': 379479,
 'yusaf': 270311,
 'hhsoffice': 141697,
 '5985': 32985,
 'fsos': 109474,
 'caned': 324502,
 'gaa': 534680,
 'iguau': 456103,
 'storiesin': 513151,
 'braziljorge': 107111,
 'iguaz': 127410,
 'kealhofer': 342134,
 'canek': 159182,
 '2116': 429919,
 'canem': 83461,
 'victorialooking': 58169,
 'martre': 111954,
 'lippert': 491344,
 'pagesolove': 33220,
 'sowell': 489254,
 'weiskopfs': 344360,
 'hedquist': 165506,
 '

In [15]:
word_count[:,519990].toarray().sum()

32

**Exercice 2-4 (optionnel) - Extraire par vous-même les vecteurs d'occurence des mots. Un bon point de départ est la fonction `sklearn.CountVectorizer`**

In [16]:
# Compléter cette cellule

## 3 - Recherche des plus proches voisins avec représentation word count

Commençons par trouver les voisins les plus proches de la page Wikipedia de **Barack Obama**. Les vecteurs d'occurence des mots (**word count**) seront utilisés pour représenter les articles et la **distance euclidienne** pour mesurer la similarité.

[class sklearn.neighbors.NearestNeighbors(*, n_neighbors=5, radius=1.0, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, n_jobs=None)](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)

**Exercice 3-1 - Quel est l'id correspondant à la page Wikipedia de barack Obama ?**

In [17]:
# Compléter cette cellule ~ 1 ligne de code

In [18]:
#wiki['name'] == 'Barack Obama'
wiki[wiki['name'] == 'Barack Obama']

,URI,name,text
id,,,
35817,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...


**Exercice 3-2 - À l'aide de scikit-learn, rechercher les 10 pages Wikipedia de personnalités les plus similaires à la page de Barack Obama. Affichez les distances et noms de personalités dans une même trame de données**

In [19]:
# Compléter cette cellule ~ 5-6 lignes de code

In [20]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='euclidean', algorithm='brute').fit(word_count)
distances, indices = model.kneighbors(word_count[35817], n_neighbors=10)

In [21]:
indices

array([[35817, 24478, 28447, 35357, 14754, 13229, 31423, 22745, 36364,
         9210]])

In [22]:
neighbors = pd.DataFrame({'distance':distances.flatten(), 'id':indices.flatten()}).set_index('id')
wiki.join(neighbors, on='id', how="right").sort_values(by='distance')[['name','distance']]

,name,distance
id,,
35817,Barack Obama,0.000000
24478,Joe Biden,33.075671
28447,George W. Bush,34.394767
35357,Lawrence Summers,36.152455
14754,Mitt Romney,36.166283
13229,Francisco Barrio,36.331804
31423,Walter Mondale,36.400549
22745,Wynn Normington Hugh-Jones,36.496575
36364,Don Bonker,36.633318


**Exercice 3-3 - Interprétez les résultats ci-dessus**

In [23]:
# Compléter cette cellule

Les 10 personnalités sont toutes des politiciens, mais à peu près la moitié d'entre elles ont des liens assez ténus avec Obama, outre le fait qu'ils sont des politiciens.

* Francisco Barrio est un homme politique mexicain et ancien gouverneur de Chihuahua.
* Walter Mondale et Don Bonker sont des démocrates qui ont fait carrière à la fin des années 1970.
* Wynn Normington Hugh-Jones est un ancien diplomate britannique et fonctionnaire du Parti libéral.
* Andy Anstett est un ancien politicien au Manitoba, au Canada.

**Exercice 3-4 - Affichez les mots les plus fréquents des pages de Barack Obama et Francisco Barrio**

Afin de pouvoir reconnaître rapidement les mots d'une grande importance, la fonction suivante permettant d'obtenir la colonne `word_count` est fournie.

In [24]:
def unpack_dict(matrix, map_index_to_word):
    table = sorted(map_index_to_word, key=map_index_to_word.get)
   
    data = matrix.data
    indices = matrix.indices
    indptr = matrix.indptr
    
    num_doc = matrix.shape[0]

    return [{k:v for k,v in zip([table[word_id] for word_id in indices[indptr[i]:indptr[i+1]] ],
                                 data[indptr[i]:indptr[i+1]].tolist())} for i in range(num_doc) ]

In [25]:
# Compléter cette cellule ~ 2 lignes de code

In [32]:
wiki['word_count'] = unpack_dict(word_count, map_index_to_word)
wiki['word_count'].iloc[35817]

{'husen': 1,
 '2012obama': 1,
 'laureateduring': 1,
 'normalize': 1,
 'brk': 1,
 'doddfrank': 1,
 'obamacare': 1,
 'reauthorization': 1,
 'reinvestment': 1,
 'rodham': 1,
 'stimulus': 1,
 'briefs': 1,
 'taxpayer': 1,
 'repeal': 1,
 'troop': 1,
 'bm': 1,
 '44th': 1,
 'proposition': 1,
 'romney': 1,
 'unconstitutional': 1,
 'mitt': 1,
 'mccain': 1,
 'gains': 1,
 'laden': 1,
 'primaries': 1,
 'recession': 1,
 'unemployment': 1,
 'osama': 1,
 'sufficient': 1,
 'libya': 1,
 'hook': 1,
 'inaugurated': 1,
 'withdrawal': 1,
 '2000in': 1,
 'urged': 1,
 'delegates': 1,
 'affordable': 1,
 'hussein': 1,
 'hillary': 1,
 'californias': 1,
 'treaty': 1,
 'honolulu': 1,
 'lengthy': 1,
 'patient': 1,
 'bin': 1,
 'debt': 1,
 'republicans': 1,
 'lgbt': 1,
 'regained': 1,
 'ask': 1,
 'organizer': 1,
 'sandy': 1,
 'keynote': 1,
 'limit': 1,
 'equality': 1,
 'gun': 1,
 '63': 1,
 'recovery': 1,
 'nobel': 1,
 'barack': 1,
 'arms': 1,
 'strike': 1,
 'cuba': 1,
 'seats': 1,
 'sworn': 1,
 'elementary': 1,
 '13th

In [59]:
wiki

,URI,name,text,word_count
id,,,,
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{'brisbaneafter': 1, 'edflhe': 1, 'aflfrom': 1..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{'maladaptation': 1, 'phasedelay': 1, '25hour'..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{'germanyover': 1, 'bluesgospel': 1, 'harpdog'..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{'fantasticrottensteiner': 1, 'waidmannsfeld':..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{'arhm': 3, 'gangstergenka': 1, 'kuhnja': 1, '..."
...,...,...,...,...
59066,<http://dbpedia.org/resource/Olari_Elts>,Olari Elts,olari elts born april 27 1971 in tallinn eston...,"{'orchestraolaris': 1, 'ivth': 1, 'nyyd': 1, '..."
59067,<http://dbpedia.org/resource/Scott_F._Crago>,Scott F. Crago,scott francis crago born july 26 1963 twin bro...,"{'procushion': 1, '5088376': 1, 'trafton': 3, ..."
59068,<http://dbpedia.org/resource/David_Cass_(footb...,David Cass (footballer),david william royce cass born 27 march 1962 in...,"{'3257': 1, '15696': 1, 'grewcock': 1, 'orient..."


**Exercice 3-5 - Créer une fonction `top_words`, permattant d'afficher les mots les plus fréquents d'une page donnée**

In [27]:
# Compléter cette cellule ~ 10 lignes de code

In [28]:
def top_words(name):
    """
    Retourne la table des mots les plus fréquents d'une page Wikipedia du jeu de données.
    """
    row = wiki[wiki['name'] == name]
    word_count_df = pd.DataFrame(row['word_count'].apply(pd.Series).stack(), columns=["count"]).droplevel(0)
    word_count_df.index.name = 'word'
    
    return word_count_df.sort_values(by='count', ascending=False)

In [56]:
def top_words(name):
    """
    Retourne la table des mots les plus fréquents d'une page Wikipedia du jeu de données.
    """
    row = wiki[wiki['name'] == name]
    
    
    return sorted(row['word_count'].iloc[0].items(), key=lambda x:x[1], reverse=True)

In [57]:
row = wiki[wiki['name'] == 'Francisco Barrio']
row['word_count'].iloc[0]

{'politicaldemocratic': 1,
 'creelin': 1,
 'satevo': 1,
 'barrioterrazas': 1,
 'itesmbarrio': 1,
 'foxbarrio': 1,
 'chihuahuas': 1,
 'clothingin': 1,
 'terrazas': 1,
 'partyduring': 1,
 'melndez': 1,
 'baeza': 1,
 'comptrollers': 1,
 'favoritism': 1,
 'groupon': 1,
 'uninterrupted': 1,
 'mayorship': 1,
 'juarez': 2,
 'jurez': 2,
 'chihuahua': 7,
 'barrio': 2,
 'pri': 2,
 'quitting': 1,
 'raped': 1,
 '20002003': 1,
 'anticorruption': 1,
 'partisan': 1,
 'ciudad': 1,
 'governorship': 1,
 'vicente': 1,
 'monterrey': 1,
 'provocative': 1,
 'surprising': 1,
 'javier': 1,
 'consequently': 1,
 'autonomous': 1,
 'deputies': 1,
 'murdered': 1,
 'secretariat': 1,
 'murders': 1,
 'arguing': 1,
 'regained': 1,
 'campaigned': 1,
 'wore': 1,
 'technological': 1,
 'santiago': 1,
 'walked': 1,
 'accounting': 1,
 'institutional': 1,
 'fernando': 1,
 'protests': 1,
 'inquiry': 1,
 'governance': 1,
 'revolutionary': 1,
 'interior': 1,
 'delivered': 1,
 'suggested': 1,
 'pan': 2,
 'affiliated': 1,
 '80s':

In [58]:
obama_words = top_words('Barack Obama')
barrio_words = top_words('Francisco Barrio')
#combined_words = obama_words.join(barrio_words, on='word', how="inner", lsuffix='_obama', rsuffix='_barrio')
#combined_words.head(10)
obama_words

[('the', 40),
 ('in', 30),
 ('and', 21),
 ('of', 18),
 ('to', 14),
 ('his', 11),
 ('obama', 9),
 ('act', 8),
 ('he', 7),
 ('a', 7),
 ('law', 6),
 ('us', 6),
 ('as', 6),
 ('was', 5),
 ('iraq', 4),
 ('control', 4),
 ('military', 4),
 ('democratic', 4),
 ('president', 4),
 ('after', 4),
 ('has', 4),
 ('for', 4),
 ('ordered', 3),
 ('response', 3),
 ('involvement', 3),
 ('senate', 3),
 ('term', 3),
 ('campaign', 3),
 ('election', 3),
 ('signed', 3),
 ('party', 3),
 ('january', 3),
 ('states', 3),
 ('2011', 3),
 ('2004', 3),
 ('united', 3),
 ('2009', 3),
 ('school', 3),
 ('american', 3),
 ('first', 3),
 ('from', 3),
 ('with', 3),
 ('nominee', 2),
 ('afghanistan', 2),
 ('domestic', 2),
 ('relief', 2),
 ('protection', 2),
 ('dont', 2),
 ('representatives', 2),
 ('illinois', 2),
 ('presidential', 2),
 ('republican', 2),
 ('primary', 2),
 ('harvard', 2),
 ('foreign', 2),
 ('policy', 2),
 ('chicago', 2),
 ('office', 2),
 ('20', 2),
 ('house', 2),
 ('november', 2),
 ('second', 2),
 ('2010', 2),
 (

## 4 - Recherche des plus proches voisins avec représentation TF-IDF

**Exercice 4 - Répétez les étapes des exercices de la partie 3 en utilisant cette fois-ci la représentation TF-IDF. Comparez avec les résultats obtenu par la représentation word count**

In [ ]:
# Compléter cette cellule ~ 14-20 lignes de code

In [ ]:
# Chargement des représentations TF-IDF
tf_idf = load_sparse_csr('../../data/people/people_wiki_tf_idf.npz')

# Recherche des 10 plus proches voisins
model_tf_idf = NearestNeighbors(metric='euclidean', algorithm='brute').fit(tf_idf)
distances, indices = model_tf_idf.kneighbors(tf_idf[35817], n_neighbors=10)

# Préparation de la trame de données des résultats
neighbors = pd.DataFrame({'distance':distances.flatten(), 'id':indices.flatten()}).set_index('id')
wiki.join(neighbors, on='id', how='right').sort_values(by='distance')[['name','distance']]

In [ ]:
# Affichage des mots les plus significatifs des deux pages
wiki['tf_idf'] = unpack_dict(tf_idf, map_index_to_word)
def top_words_tf_idf(name):
    row = wiki[wiki['name'] == name]
    tf_idf_df = pd.DataFrame(row['tf_idf'].apply(pd.Series).stack(), columns=["weight"]).droplevel(0)
    tf_idf_df.index.name = 'word'
    return tf_idf_df.sort_values(by='weight', ascending=False)

obama_words = top_words_tf_idf('Barack Obama')
barrio_words = top_words_tf_idf('Francisco Barrio')
combined_words = obama_words.join(barrio_words, on='word', how="inner", lsuffix='_obama', rsuffix='_barrio')
combined_words.head(10)

### Fin de l'atelier 02-02-A1